<a href="https://colab.research.google.com/github/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/An%C3%A1lise_e_Explora%C3%A7%C3%A3o_de_Dados_Cr%C3%A9dito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploração e análise de dados de crédito com SQL

Sou analista de negócios e dados em um banco no interior do Brasil, após a última reunião para informar o desempenho foi constado uma queda na receita de 23% YoY no último trimestre.

Após uma análise mais detalhada se observou que o produto Cartão de Crédito era o que apresentava maior queda de receita.
Foi extraído da base de dados de cadastro uma amostra com as seguintes informações dos clientes:

* idade = idade do cliente

* sexo = sexo do cliente (F ou M)

* dependentes = número de dependentes do cliente

* escolaridade = nível de escolaridade dos clientes

* salario_anual = faixa salarial do cliente

* tipo_cartao = tipo de cartão do cliente

* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses

* iteracoes_12m = quantidade de iterações/transações nos últimos 12 meses

* meses_inativo_12m = quantidade de meses que o cliente ficou inativo

* limite_credito = limite de crédito do cliente

* valor_transacoes_12m = valor das transações dos últimos 12 meses

* qtd_transacoes_12m = quantidade de transações dos últimos 12 meses




A tabela foi criada no AWS Athena junto com o S3 Bucket com dados disponibilizados em: https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset


# Exploração de dados:

**1) Verificar a quantidade de informações em nossa base**

Query: SELECT count(*) FROM credito;

> Resposta: 2564

**2) Como são os dados**

Query: SELECT * FROM credito LIMIT 10;

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/query2.png?raw=true)

> Inicialmente podemos notar que há informações nulas com o valor (na)

**3) Verificando os tipos de cada dado**

Query: DESCRIBE credito;

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/query3.png?raw=true)



**4) Verificando em quais colunas há valores nulos para tratarmos depois**

    4.1) Verificando tipos de escolaridade distintas

Query: SELECT DISTINCT escolaridade FROM credito;

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/query4.png?raw=true)

    4.2) Verificando tipos de estado civil distintos

Query: SELECT DISTINCT estado_civil FROM credito;

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/query4-2.png?raw=true)

    4.3) Verificando salário anual distintos

SELECT DISTINCT salario_anual FROM credito;

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/query4-3.png?raw=true)

    4.4) Verificando tipos de cartões distintos

Query: SELECT DISTINCT tipo_cartao FROM credito;

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/query4-4.png?raw=true)

> Nessa coluna não será necessário tratar nulos (na)

# Análise de dados

### Ciente dos tipos de dados que temos, vamos começar a analisar e fazer perguntas sobre o perfil dos nossos clientes.

* ## Qual a proporção de clientes para cada faixa de renda?

Query: SELECT COUNT(*) AS contagem, salario_anual FROM credito GROUP BY salario_anual ORDER BY contagem

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/div-faixa-renda.png?raw=true)

> Observamos que no banco de dados é informado não valor exato do salario anual, mas sim em faixas salariais

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/div-faixa-renda-grafico.png?raw=true)

> No gráfico percebemos que a maior faixa salarial é de "menos que $40K" represetando 27,34% dos clientes





* ## Qual a faixa salarial que tem o maior valor de transações?
  

Query: SELECT salario_anual, SUM(valor_transacoes_12m) AS total_transacoes, AVG(valor_transacoes_12m) AS media_transacoes, SUM(iteracoes_12m) AS contagem_iteracoes FROM credito WHERE salario_anual != 'na' GROUP BY salario_anual

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/salario-transacoes-total-med-cont.png?raw=true)

  > Os clientes da faixa salarial de "menos de $40K" representam  aproximadamente 38% do volume em transações



* ##  O tipo do cartão e o limite de crédito podem influenciar a média de gastos?
  

Query: SELECT tipo_cartao, COUNT(*) AS n_clientes, AVG(limite_credito) AS media_limite, AVG(valor_transacoes_12m) AS media_transacoes, SUM(iteracoes_12m) AS n_iteracoes FROM credito WHERE salario_anual != 'na' GROUP BY tipo_cartao ORDER BY media_transacoes DESC

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/cartao-medlimite-medtransacoes.png?raw=true)

> A resposta é sim, nota-se grande disparedade na quantidade de clientes com cartão blue, e em relação ao cartão silver há uma pequena diferença na média de valor nas transações, mas o limite é 3x maior em relação a média dos clientes com cartão blue.

## Pode-se concluir que uma estratégia para aumentar a receita do banco seria promover um upgrade de cartão para os clientes mais qualificados para um cartão silver


> Quais seriam os clientes com cartão blue mais qualificados ? Em uma análise anterior identificamos que o valor médios das transações por faixa salarial é maior em clientes  que ganham "menos de $40k" se compararmos com os valores de faixas salarias maiores.





* ## Analisando mais o perfil do cliente com cartão blue

Query: SELECT salario_anual, COUNT(*) AS n_clientes, SUM(iteracoes_12m) AS n_transacoes, AVG(valor_transacoes_12m) AS valor_medio_transacoes, MAX (limite_credito) AS limite_credito_max, MIN(limite_credito) AS limite_credito_min FROM credito WHERE tipo_cartao = 'blue' AND salario_anual != 'na' GROUP BY salario_anual

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/clientes-cart%C3%A3o-blue.png?raw=true)

> Mesmo com um limite menor, clientes da faixa salarial "menos que $40K" ainda possuem a média de valor gasto acima do que outras faixas.


* ##  Filtrando listas de possiveis clientes para serem befenificados pelo upgrade de cartão

Query: SELECT * FROM credito WHERE limite_credito > (SELECT AVG(limite_credito) FROM credito)  AND iteracoes_12m > (SELECT AVG(iteracoes_12m) FROM credito)  AND tipo_cartao = 'blue';

![](https://github.com/fabiobaroliveira/analise_exploratoria_de_dados/blob/main/projeto_sql/upgrade_clientes_silver.png?raw=true)

> Teríamos uma lista de 380 clientes que poderiam ser beneficiados com um upgrade para o cartão silver, esse clientes estariam mais qualificados devido a já terem um limite e numeros de transacoes acima da média.


# Conclusão

* A maior faixa salarial é representado pelos clientes que tem salarial anual "menos que 40k", cerca de 27,3%.
* Aproximadamente 38% das transações é realizada pela mesma faixa salarial citada anteriormente.
* Foi observado que 86,9% dos clientes do banco possui o cartão blue.
* Em comparação os clientes com cartão blue têm valores médios de gastos proxímos aos dos clientes com cartão silver.
* Uma solução para a queda de receita seria oferecer um upgrade de cartão para clientes do cartão blue para cartão silver que possuem valor de transações igual ou maior a clientes desse cartão.

